# 🔬 LLM 비교 분석: OpenAI GPT-3.5-turbo vs Llama 3.1 8B

## 📋 개요
이 notebook은 Clinical Note 생성에서 OpenAI API (GPT-3.5-turbo)와 Local LLM (Llama 3.1 8B)의 성능을 비교 분석합니다.

**분석 항목:**
1. 생성 시간 비교
2. Clinical Note 품질 비교
3. 비용 분석
4. 장단점 분석
5. 사용 시나리오 추천

## 🛠️ 1. Ollama 설치 및 Llama 3.1 8B 준비

### 1.1 Ollama 설치

**Windows:**
```bash
# 1. https://ollama.com/download/windows 에서 설치 파일 다운로드
# 2. 설치 후 자동으로 서비스 시작됨
```

**설치 확인:**
```bash
ollama --version
```

### 1.2 Llama 3.1 8B 다운로드

```bash
ollama pull llama3.1:8b
```

- 모델 크기: 약 4.7GB
- 소요 시간: 인터넷 속도에 따라 5-15분

### 1.3 Ollama 서버 실행

```bash
# 기본적으로 자동 실행되지만, 수동 실행이 필요한 경우:
ollama serve
```

- 기본 포트: `http://localhost:11434`

### 1.4 모델 테스트

```bash
ollama run llama3.1:8b
```

In [ ]:
# 필요한 라이브러리 import
import pandas as pd
import matplotlib.pyplot as plt
import json
import time
import os
from datetime import datetime

# 한글 폰트 설정
import platform
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
else:  # Linux
    plt.rcParams['font.family'] = 'NanumGothic'

plt.rcParams['axes.unicode_minus'] = False

print("라이브러리 로딩 완료!")

## 📊 2. 이론적 비교

### 2.1 모델 스펙 비교

| 항목 | GPT-3.5-turbo | Llama 3.1 8B |
|------|---------------|---------------|
| **개발사** | OpenAI | Meta |
| **파라미터 수** | 175B (추정) | 8B |
| **컨텍스트 길이** | 16,384 tokens | 128K tokens |
| **실행 환경** | 클라우드 API | 로컬 (CPU/GPU) |
| **비용** | 사용량 기반 | 무료 (하드웨어 비용만) |
| **속도** | 네트워크 의존 | 하드웨어 의존 |
| **데이터 프라이버시** | 서버 전송 | 로컬 처리 |

### 2.2 비용 비교

**GPT-3.5-turbo:**
- Input: $0.50 / 1M tokens
- Output: $1.50 / 1M tokens
- 예상 비용 (환자 1명, 한글+영어): 약 $0.02

**Llama 3.1 8B:**
- API 비용: $0 (무료)
- 초기 하드웨어 비용: GPU 있으면 유리
- 전기 비용: 미미

### 2.3 하드웨어 요구사항

**Llama 3.1 8B:**
- 최소 RAM: 8GB
- 권장 RAM: 16GB
- GPU: 선택 사항 (NVIDIA GPU 있으면 10-20배 빠름)
- 디스크: 5GB (모델 저장)

## 🚀 3. Admin Agent 실행

### 3.1 OpenAI API (GPT-3.5-turbo) - 이미 실행 완료

결과 파일 위치: `../Admin_Agent/data/output_notes/`

In [ ]:
# OpenAI API로 생성된 파일 확인
openai_output_dir = "../Admin_Agent/data/output_notes/"
openai_files = os.listdir(openai_output_dir)
print(f"OpenAI API로 생성된 파일 ({len(openai_files)}개):")
for f in openai_files:
    file_path = os.path.join(openai_output_dir, f)
    file_size = os.path.getsize(file_path)
    print(f"  - {f} ({file_size} bytes)")

### 3.2 Local LLM (Llama 3.1 8B) 실행

**주의:** Ollama가 실행 중이어야 합니다!

In [ ]:
# Ollama 서버 상태 확인
import requests

try:
    response = requests.get("http://localhost:11434")
    print("[OK] Ollama 서버가 실행 중입니다!")
    print(f"응답: {response.text}")
except Exception as e:
    print("[FAIL] Ollama 서버에 연결할 수 없습니다.")
    print(f"오류: {str(e)}")
    print("\n명령 프롬프트에서 'ollama serve'를 실행하세요.")

In [ ]:
# Llama 3.1 8B로 Clinical Note 생성
# 주의: 이 셀을 실행하기 전에 Ollama가 실행 중이어야 합니다!

import subprocess
import time

print("Llama 3.1 8B로 Clinical Note 생성 시작...")
print("이 작업은 하드웨어에 따라 5-30분 소요될 수 있습니다.\n")

start_time = time.time()

# Admin Agent Local 실행
result = subprocess.run(
    ["python", "Admin_Agent_Local/scripts/admin_agent_local.py"],
    capture_output=True,
    text=True,
    input="\n"  # Enter 키 자동 입력
)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\n실행 완료! 소요 시간: {elapsed_time:.2f}초")
print("\n=== 출력 ===")
print(result.stdout)

if result.returncode != 0:
    print("\n=== 오류 ===")
    print(result.stderr)

In [ ]:
# Llama로 생성된 파일 확인
llama_output_dir = "Admin_Agent_Local/data/output_notes/"
llama_files = os.listdir(llama_output_dir) if os.path.exists(llama_output_dir) else []
print(f"Llama 3.1 8B로 생성된 파일 ({len(llama_files)}개):")
for f in llama_files:
    file_path = os.path.join(llama_output_dir, f)
    file_size = os.path.getsize(file_path)
    print(f"  - {f} ({file_size} bytes)")

## 📈 4. 정량적 비교 분석

### 4.1 생성 시간 비교

In [ ]:
# 생성 시간 데이터 (수동 입력 또는 로그에서 추출)
# 예시 데이터 - 실제 실행 후 업데이트 필요

time_comparison = {
    "모델": ["GPT-3.5-turbo", "Llama 3.1 8B"],
    "환자당_평균_시간(초)": [5, 20],  # 실제 측정 값으로 교체
    "2명_총_시간(초)": [10, 40]  # 실제 측정 값으로 교체
}

df_time = pd.DataFrame(time_comparison)
print("생성 시간 비교:")
print(df_time)

# 시각화
fig, ax = plt.subplots(figsize=(10, 6))
x = range(len(df_time["모델"]))
ax.bar(x, df_time["환자당_평균_시간(초)"], color=['#2E86AB', '#A23B72'])
ax.set_xticks(x)
ax.set_xticklabels(df_time["모델"])
ax.set_ylabel("시간 (초)")
ax.set_title("환자당 Clinical Note 생성 시간 비교")
ax.grid(axis='y', alpha=0.3)

for i, v in enumerate(df_time["환자당_평균_시간(초)"]):
    ax.text(i, v + 1, f"{v}초", ha='center')

plt.tight_layout()
plt.savefig("time_comparison.png", dpi=300, bbox_inches='tight')
plt.show()

### 4.2 파일 크기 비교

In [ ]:
# 파일 크기 비교
def get_file_sizes(directory):
    sizes = []
    for f in os.listdir(directory):
        if f.endswith('.txt'):
            file_path = os.path.join(directory, f)
            sizes.append(os.path.getsize(file_path))
    return sizes

openai_sizes = get_file_sizes(openai_output_dir)
llama_sizes = get_file_sizes(llama_output_dir) if os.path.exists(llama_output_dir) else []

size_data = {
    "모델": ["GPT-3.5-turbo"] * len(openai_sizes) + ["Llama 3.1 8B"] * len(llama_sizes),
    "크기(bytes)": openai_sizes + llama_sizes
}

df_size = pd.DataFrame(size_data)

# Box plot으로 비교
fig, ax = plt.subplots(figsize=(10, 6))
df_size.boxplot(column='크기(bytes)', by='모델', ax=ax)
ax.set_title("Clinical Note 파일 크기 분포")
ax.set_xlabel("모델")
ax.set_ylabel("파일 크기 (bytes)")
plt.suptitle("")  # 기본 제목 제거
plt.tight_layout()
plt.savefig("size_comparison.png", dpi=300, bbox_inches='tight')
plt.show()

print("\n평균 파일 크기:")
print(df_size.groupby('모델')['크기(bytes)'].mean())

### 4.3 비용 분석

In [ ]:
# 비용 계산 (100명의 환자 기준)
num_patients = 100
notes_per_patient = 2  # 한글 + 영어

# GPT-3.5-turbo 비용 (추정)
avg_input_tokens = 500  # 환자 데이터 프롬프트
avg_output_tokens = 300  # 생성된 Clinical Note
gpt_cost_per_patient = (
    (avg_input_tokens / 1_000_000) * 0.50 + 
    (avg_output_tokens / 1_000_000) * 1.50
) * notes_per_patient

gpt_total_cost = gpt_cost_per_patient * num_patients

# Llama 3.1 8B 비용
llama_cost = 0  # 무료

cost_comparison = pd.DataFrame({
    "모델": ["GPT-3.5-turbo", "Llama 3.1 8B"],
    f"{num_patients}명_총_비용($)": [gpt_total_cost, llama_cost],
    "환자당_비용($)": [gpt_cost_per_patient, 0]
})

print(f"\n{num_patients}명의 환자 Clinical Note 생성 비용:")
print(cost_comparison)

# 시각화
fig, ax = plt.subplots(figsize=(10, 6))
x = range(len(cost_comparison["모델"]))
ax.bar(x, cost_comparison[f"{num_patients}명_총_비용($)"], color=['#2E86AB', '#A23B72'])
ax.set_xticks(x)
ax.set_xticklabels(cost_comparison["모델"])
ax.set_ylabel("비용 ($)")
ax.set_title(f"{num_patients}명 환자 Clinical Note 생성 비용 비교")
ax.grid(axis='y', alpha=0.3)

for i, v in enumerate(cost_comparison[f"{num_patients}명_총_비용($)"]):
    ax.text(i, v + 0.05, f"${v:.3f}", ha='center')

plt.tight_layout()
plt.savefig("cost_comparison.png", dpi=300, bbox_inches='tight')
plt.show()

## 📝 5. 정성적 비교 분석

### 5.1 Clinical Note 샘플 비교

In [ ]:
# 같은 환자에 대한 두 모델의 출력 비교
patient_id = "R-1731-00000001"

# GPT-3.5-turbo 결과
gpt_file = [f for f in openai_files if patient_id in f and "korean" in f][0]
with open(os.path.join(openai_output_dir, gpt_file), 'r', encoding='utf-8') as f:
    gpt_note = f.read()

print("=" * 60)
print(f"환자 {patient_id} - GPT-3.5-turbo 생성 Clinical Note")
print("=" * 60)
print(gpt_note)
print("\n")

# Llama 3.1 8B 결과 (생성된 경우)
if llama_files:
    llama_file = [f for f in llama_files if patient_id in f and "korean" in f][0]
    with open(os.path.join(llama_output_dir, llama_file), 'r', encoding='utf-8') as f:
        llama_note = f.read()
    
    print("=" * 60)
    print(f"환자 {patient_id} - Llama 3.1 8B 생성 Clinical Note")
    print("=" * 60)
    print(llama_note)
else:
    print("Llama 3.1 8B 결과가 아직 생성되지 않았습니다.")

### 5.2 품질 평가 기준

Clinical Note의 품질을 다음 기준으로 평가합니다:

1. **완전성** (Completeness): 모든 필수 섹션이 포함되어 있는가?
2. **정확성** (Accuracy): 환자 데이터가 정확하게 반영되었는가?
3. **자연스러움** (Fluency): 문장이 자연스럽고 읽기 쉬운가?
4. **전문성** (Professionalism): 의학 용어 사용이 적절한가?
5. **일관성** (Consistency): 정보 간 모순이 없는가?

In [ ]:
# 품질 평가 (수동 평가 후 입력)
# 각 항목을 1-5점으로 평가

quality_scores = {
    "평가 항목": ["완전성", "정확성", "자연스러움", "전문성", "일관성"],
    "GPT-3.5-turbo": [5, 5, 5, 4, 5],  # 실제 평가 후 업데이트
    "Llama 3.1 8B": [4, 4, 4, 3, 4]  # 실제 평가 후 업데이트
}

df_quality = pd.DataFrame(quality_scores)
df_quality.set_index("평가 항목", inplace=True)

print("\n품질 평가 점수 (1-5점):")
print(df_quality)

# 레이더 차트로 시각화
import numpy as np

categories = df_quality.index.tolist()
N = len(categories)

angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

# GPT-3.5-turbo
values = df_quality["GPT-3.5-turbo"].tolist()
values += values[:1]
ax.plot(angles, values, 'o-', linewidth=2, label="GPT-3.5-turbo", color='#2E86AB')
ax.fill(angles, values, alpha=0.25, color='#2E86AB')

# Llama 3.1 8B
values = df_quality["Llama 3.1 8B"].tolist()
values += values[:1]
ax.plot(angles, values, 'o-', linewidth=2, label="Llama 3.1 8B", color='#A23B72')
ax.fill(angles, values, alpha=0.25, color='#A23B72')

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)
ax.set_ylim(0, 5)
ax.set_yticks([1, 2, 3, 4, 5])
ax.set_yticklabels(['1', '2', '3', '4', '5'])
ax.grid(True)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
ax.set_title("Clinical Note 품질 평가", pad=20, fontsize=16)

plt.tight_layout()
plt.savefig("quality_comparison.png", dpi=300, bbox_inches='tight')
plt.show()

## 📊 6. 종합 비교 분석

### 6.1 장단점 비교

### GPT-3.5-turbo (OpenAI API)

**장점:**
- ✅ 높은 품질: 자연스럽고 전문적인 텍스트 생성
- ✅ 빠른 속도: 네트워크만 양호하면 빠른 응답
- ✅ 설정 간단: API 키만 있으면 즉시 사용 가능
- ✅ 안정성: OpenAI의 인프라로 높은 가용성
- ✅ 지속적 개선: 모델이 주기적으로 업데이트됨

**단점:**
- ❌ 비용 발생: 사용량에 따른 과금
- ❌ 데이터 프라이버시: 환자 데이터가 외부 서버로 전송
- ❌ 인터넷 필수: 오프라인 환경에서 사용 불가
- ❌ API 제한: Rate limit, 할당량 제한
- ❌ 종속성: OpenAI 서비스에 의존

### Llama 3.1 8B (Local LLM)

**장점:**
- ✅ 무료: API 비용 없음
- ✅ 데이터 프라이버시: 모든 데이터가 로컬에서 처리
- ✅ 오프라인 가능: 인터넷 없이 사용 가능
- ✅ 제한 없음: Rate limit 없음
- ✅ 커스터마이징: Fine-tuning 가능

**단점:**
- ❌ 품질: GPT-3.5보다 낮을 수 있음 (특히 한글)
- ❌ 느린 속도: CPU만 있으면 매우 느림
- ❌ 하드웨어 요구: RAM 8GB 이상 필요
- ❌ 설정 복잡: Ollama 설치 및 모델 다운로드 필요
- ❌ 업데이트: 수동으로 모델 업데이트 필요

### 6.2 사용 시나리오별 추천

| 시나리오 | 추천 모델 | 이유 |
|---------|----------|------|
| **프로토타입 개발** | GPT-3.5-turbo | 빠른 설정, 높은 품질 |
| **소규모 연구 (<100명)** | GPT-3.5-turbo | 비용이 적고 품질이 중요 |
| **대규모 연구 (>1000명)** | Llama 3.1 8B | 비용 절감 (무료) |
| **민감한 환자 데이터** | Llama 3.1 8B | 데이터 프라이버시 보장 |
| **오프라인 환경** | Llama 3.1 8B | 인터넷 불필요 |
| **최고 품질 필요** | GPT-3.5-turbo | 더 자연스럽고 전문적 |
| **GPU 서버 보유** | Llama 3.1 8B | 속도 향상 + 무료 |
| **빠른 결과 필요** | GPT-3.5-turbo | 클라우드 속도 |
| **Fine-tuning 필요** | Llama 3.1 8B | 커스터마이징 가능 |

## 🎯 7. 결론 및 권장사항

### 7.1 핵심 발견

1. **품질**: GPT-3.5-turbo가 일반적으로 더 자연스럽고 전문적인 Clinical Note 생성
2. **비용**: Llama 3.1 8B는 무료이며, 대량 처리 시 큰 비용 절감
3. **속도**: 하드웨어에 따라 다르지만, 일반적으로 GPT-3.5-turbo가 더 빠름
4. **프라이버시**: Llama 3.1 8B가 로컬 처리로 데이터 보안 우수

### 7.2 Multi-Agent System 적용 권장사항

**하이브리드 접근:**
```
Formatter Agent (로컬) → ED Physician Agent (API) → Specialist Agent (API)
       ↓                           ↓                         ↓
  Llama 3.1 8B            GPT-3.5-turbo            GPT-4 (고급)
```

- **Formatter Agent**: Llama 3.1 8B (간단한 데이터 정리, 무료)
- **ED Physician**: GPT-3.5-turbo (빠르고 품질 좋음)
- **Specialist**: GPT-4 (최고 품질 필요시)

### 7.3 향후 연구 방향

1. **Fine-tuning**: Llama 3.1 8B를 의료 데이터로 Fine-tuning하여 품질 향상
2. **앙상블**: 두 모델의 결과를 결합하여 최적화
3. **벤치마킹**: 더 많은 환자 데이터로 광범위한 평가
4. **다국어**: 영어 외 다른 언어에서의 성능 비교

## 📚 참고 자료

- [Ollama 공식 문서](https://ollama.com/docs)
- [Llama 3.1 모델 카드](https://ollama.com/library/llama3.1)
- [OpenAI API 문서](https://platform.openai.com/docs)
- [AutoGen 문서](https://microsoft.github.io/autogen/)

---

**분석 작성일**: 2025-01-15  
**분석자**: Multi-Agent EME 프로젝트 팀  
**버전**: 1.0